In [1]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from envs.humanoid import HumanoidTracking
import numpy as np
import mediapy as media
import jax.numpy as jp
import mujoco
from dm_control import mjcf

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# CMU Humanoid env

In [2]:
_CMU_MOCAP_JOINTS = (
    'lfemurrz', 'lfemurry', 'lfemurrx', 'ltibiarx', 'lfootrz', 'lfootrx',
    'ltoesrx', 'rfemurrz', 'rfemurry', 'rfemurrx', 'rtibiarx', 'rfootrz',
    'rfootrx', 'rtoesrx', 'lowerbackrz', 'lowerbackry', 'lowerbackrx',
    'upperbackrz', 'upperbackry', 'upperbackrx', 'thoraxrz', 'thoraxry',
    'thoraxrx', 'lowerneckrz', 'lowerneckry', 'lowerneckrx', 'upperneckrz',
    'upperneckry', 'upperneckrx', 'headrz', 'headry', 'headrx', 'lclaviclerz',
    'lclaviclery', 'lhumerusrz', 'lhumerusry', 'lhumerusrx', 'lradiusrx',
    'lwristry', 'lhandrz', 'lhandrx', 'lfingersrx', 'lthumbrz', 'lthumbrx',
    'rclaviclerz', 'rclaviclery', 'rhumerusrz', 'rhumerusry', 'rhumerusrx',
    'rradiusrx', 'rwristry', 'rhandrz', 'rhandrx', 'rfingersrx', 'rthumbrz',
    'rthumbrx')
len(_CMU_MOCAP_JOINTS)

56

Checking the body positions and make sure they are aligned

In [3]:
root = mjcf.from_path("./assets/humanoid_CMU_V2019.xml")

physics = mjcf.Physics.from_mjcf_model(root).model.ptr

physics.body_pos.shape

physics.name_bodyadr

mj_model = mujoco.MjModel.from_xml_path("./assets/humanoid_CMU_V2019.xml")

params = np.array([
    # 'world',
    # 'root',
    'lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot',
    'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus',
    'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist',
    'rhand', 'rfingers', 'rthumb'])

jp.array([mujoco.mj_name2id(mj_model, mujoco.mju_str2Type("body"), body) for body in params])

Array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], dtype=int32)

In [4]:
params = {
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 6,
    "clip_path": "clips/test_traj.p",
    "body_names":
    ['lhipjoint', 'lfemur', 'ltibia', 'lfoot', 'ltoes', 'rhipjoint', 'rfemur', 'rtibia', 'rfoot',
    'rtoes', 'lowerback', 'upperback', 'thorax', 'lowerneck', 'upperneck', 'head', 'lclavicle', 'lhumerus',
    'lradius', 'lwrist', 'lhand', 'lfingers', 'lthumb', 'rclavicle', 'rhumerus', 'rradius', 'rwrist',
    'rhand', 'rfingers', 'rthumb'],
    "joint_names":
    ['root', 'lowerbackrz', 'lowerbackry', 'lowerbackrx', 'upperbackrz', 'upperbackry', 'upperbackrx', 'thoraxrz',
     'thoraxry', 'thoraxrx', 'lowerneckrz', 'lowerneckry', 'lowerneckrx', 'upperneckrz', 'upperneckry', 'upperneckrx', 'headrz', 'headry',
     'headrx', 'lclaviclerz', 'lclaviclery', 'lhumerusrz', 'lhumerusry', 'lhumerusrx', 'lradiusrx', 'lwristry', 'lhandrz', 'lhandrx', 'lfingersrx',
     'lthumbrz', 'lthumbrx', 'rclaviclerz', 'rclaviclery', 'rhumerusrz', 'rhumerusry', 'rhumerusrx', 'rradiusrx', 'rwristry', 'rhandrz',
     'rhandrx', 'rfingersrx', 'rthumbrz', 'rthumbrx', 'lfemurrz', 'lfemurry', 'lfemurrx', 'ltibiarx', 'lfootrz', 'lfootrx',
     'ltoesrx', 'rfemurrz', 'rfemurry', 'rfemurrx', 'rtibiarx', 'rfootrz', 'rfootrx', 'rtoesrx']
}

In [5]:
env_brax = HumanoidTracking(params)

In [6]:
env_brax.sys.nv # nq=63, nv=62

62

In [7]:
lst =[]
for n in range(100):
      joint = mujoco.mj_id2name(env_brax.sys.mj_model, 
                          mujoco.mju_str2Type("joint"), 
                          n)
      lst.append(joint)
# root joint is a free joint in any direction (adds extra dimensions), rotation locks, quaternion
# something for full range of motion

lst[:5]

['root', 'lfemurrz', 'lfemurry', 'lfemurrx', 'ltibiarx']

In [8]:
state = env_brax.reset(jax.random.PRNGKey(0))

[-6.7749232e-02  8.6231794e-06 -1.7833591e-02 -1.0716664e-01
  3.4846365e-05 -2.3776948e-02  1.6838254e-01 -2.8296500e-02
  2.9513806e-02  8.9463890e-03]


## Stepping brax environment

In [9]:
# key = random.PRNGKey(0)
# jit_step = jax.jit(env_brax.step)
# jit_reset = jax.jit(env_brax.reset)

In [10]:
# next = jit_reset(key)
# print("starting rollout")
# mu = 0
# sigma = .1
# rollout = []
# for _ in range(500):
#     _, key = jax.random.split(key)
#     next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
#     rollout.append(next)

In [11]:
# import mediapy as media
# import os

# os.environ["MUJOCO_GL"] = "glfw"
# rollout_data = [s.pipeline_state for s in rollout]

# video = env.render(rollout_data, camera='side')

# media.show_video(video, fps=1.0 / env.dt)

# COMIC Validation Observation

In [12]:
from dm_control import composer
from dm_control.locomotion import arenas
from dm_control.locomotion import walkers
from walker import Rat
from cmu_humanoid import CMUHumanoidPositionControlled
from dm_control.locomotion.mocap import props
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import types
from dm_control.utils import io as resources
import os
import h5py
import pickle

maybe reset doesn't replace values in dm control? instantiate it in init? The clip is actually instantiated in init with flag `always_init_at_clip_start=True`

In [13]:
# Define the path to your motion capture data file
file_name = 'clips/test_traj.h5' #'clips/all_snips_250.h5'
current_directory = os.getcwd()
TEST_FILE_PATH = os.path.join(current_directory, file_name)

with h5py.File(TEST_FILE_PATH, 'r') as f:
    dataset_keys = tuple(f.keys())
    dataset = types.ClipCollection(ids=dataset_keys,)

# Set up the mocap tracking task
task = tracking.MultiClipMocapTracking(
    walker=CMUHumanoidPositionControlled,
    arena=arenas.Floor(),
    ref_path=resources.GetResourceFilename(TEST_FILE_PATH),
    ref_steps=(1, 2, 3, 4, 5),
    min_steps=1,
    dataset=dataset,
    reward_type='comic',
    always_init_at_clip_start=True
)

# Initialize the environment
env = composer.Environment(task=task)
reset = env.reset()

# action_spec = env.action_spec()
# env.step(np.zeros(action_spec.shape))

In [14]:
reset.observation['walker/reference_rel_bodies_pos_global'].shape

(1, 450)

In [15]:
reset.observation['walker/reference_rel_root_pos_local'].shape

(1, 15)

In [16]:
reset.observation['walker/reference_rel_joints'].shape # 280 / 5 = 56 makes sense

(1, 280)

If the joints in brax environment is indexed with shape mismatch, no warning is given, `np` does not compain and dimension becomes (56,57) and give a flattened dimension of 3000ish, which is not the correct one.

In [17]:
env.task._walker_features.keys()

dict_keys(['position', 'quaternion', 'joints', 'center_of_mass', 'end_effectors', 'appendages', 'body_positions', 'body_quaternions', 'velocity', 'angular_velocity', 'joints_velocity'])

In [18]:
[item for item in reset.observation]

['reference_props_pos_global',
 'reference_props_quat_global',
 'walker/actuator_activation',
 'walker/appendages_pos',
 'walker/body_height',
 'walker/end_effectors_pos',
 'walker/joints_pos',
 'walker/joints_vel',
 'walker/sensors_accelerometer',
 'walker/sensors_force',
 'walker/sensors_gyro',
 'walker/sensors_torque',
 'walker/sensors_touch',
 'walker/sensors_velocimeter',
 'walker/world_zaxis',
 'walker/reference_rel_joints',
 'walker/reference_rel_bodies_pos_global',
 'walker/reference_rel_bodies_quats',
 'walker/reference_rel_bodies_pos_local',
 'walker/reference_ego_bodies_quats',
 'walker/reference_rel_root_quat',
 'walker/reference_rel_root_pos_local',
 'walker/reference_appendages_pos',
 'walker/clip_id',
 'walker/velocimeter_control',
 'walker/gyro_control',
 'walker/joints_vel_control',
 'walker/time_in_clip']

In [19]:
check_lst = ['walker/reference_rel_bodies_pos_local',
             'walker/reference_rel_bodies_pos_global',
             'walker/reference_rel_root_pos_local',
             'walker/reference_rel_joints'
            ]

COMIC_lst = reset.observation[check_lst[0]]
for check in check_lst[1:]:
    COMIC_lst = np.concatenate((COMIC_lst, reset.observation[check]), axis=1)

COMIC_lst

array([[ 7.01153267e-06,  3.45142537e-02, -1.07003822e-02, ...,
        -2.17547102e-01,  9.40013080e-05,  1.82440521e-03]])

In [20]:
state.obs.shape

(1195,)

In [21]:
COMIC_lst.shape

(1, 1195)

## Check qpos matching: position is first 3 initilizatio same

In [22]:
# env.task._clip_reference_features

In [23]:
env.physics.data.qpos

array([-1.62284346e-01, -1.31436590e-03,  1.05003126e-01,  6.97367560e-01,
        1.15868698e-01,  1.17780633e-01,  6.97409961e-01,  1.65478135e-01,
        1.16416855e-03, -1.04330490e+00,  1.77301367e+00, -3.78864520e-01,
        2.79833845e-01, -5.74850191e-01, -1.61614418e-01,  1.27503007e-02,
       -1.05328784e+00,  1.78896173e+00,  3.79252564e-01,  2.97015613e-01,
       -5.87794583e-01, -1.66355263e-03,  1.59217071e-03,  1.05798693e-01,
       -3.02797635e-04,  1.22021646e-03,  1.56459237e-01,  1.25558953e-03,
        6.28698539e-04,  1.88331063e-01,  2.82232509e-03,  1.90836430e-03,
        4.62503744e-01, -2.89387808e-04, -1.42082924e-03,  3.24895456e-01,
        1.38906628e-05, -2.39701695e-04,  2.10732833e-01, -8.07062728e-03,
       -4.47694166e-02,  1.92670023e-01, -1.37834500e-01,  2.29507658e-01,
        1.37886604e+00,  1.56205859e+00,  2.91758146e-03,  1.60174583e-03,
        7.81591598e-01,  2.60932165e-04,  7.81326703e-01,  4.84373197e-03,
        4.55006027e-02, -

In [24]:
state.pipeline_state.qpos

Array([0.        , 0.        , 0.94      , 0.46075296, 0.536383  ,
       0.536383  , 0.46075296, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ], dtype=float32)

## Each Transformation Checking (body index check)
it is probably a ordering issue? al;l the numbers looks the same, but the ordering may be what is causing the issue

In [25]:
time_steps = env.task._time_step + env.task._ref_steps
obs = env.task._walker.transform_vec_to_egocentric_frame(env.physics, (env.task._clip_reference_features['body_positions'][time_steps] -
                  env.task._walker_features['body_positions'])[:, env.task._body_idxs])
obs_flattened = np.concatenate([o.flatten() for o in obs])

env.task._walker_features['body_positions'].flatten()[:10]

array([-0.16228435, -0.00131437,  0.10500313, -0.05942653,  0.10071903,
        0.12155984,  0.22211797,  0.20225642,  0.39432953,  0.38291664])

Both from brax environments:
1. Sending to brax then grab body positions `xpos` flattened:

    ```python
    [ 0.02206819 -0.38698062  0.05154771  0.03503342  0.10193701  0.8417851
    -0.02235974  0.24043609  0.46561432 -0.07985894]
    ```

2. Grabbing body positions then sending into brax `xpos` flattened:

    ```python
    [ 0.          0.          0.94        0.03503342  0.10193701  0.8417851
    -0.02235974  0.24043609  0.46561432 -0.07985894]
    ```

Brax environment instantiation does aweird reordering that distort the original indexes

qpos values are the same but xpos are different, body position indexing wrong

In [26]:
env_brax._body_locations

Array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], dtype=int32)

positions grabbing now equivalence with the one on the top directly calling from notebook

## Each Transformation Checking (check ref_traj as well)

In [27]:
env.task._clip_reference_features['body_positions'].flatten()[:10]

array([-0.16228435, -0.00131437,  0.10500313, -0.05942653,  0.10071903,
        0.12155984,  0.22211797,  0.20225642,  0.39432953,  0.38291664])

In [28]:
env_brax._ref_traj.body_positions.flatten()[:10]

Array([ 0.        ,  0.        ,  0.94      ,  0.03503343,  0.101937  ,
        0.8417851 , -0.02235977,  0.24043609,  0.46561426, -0.07985898],      dtype=float32)

Similar as well, but...

There is a mismatch where there is a extra step from `_ref_traj.body_positions` to `_walker_features['body_positions']`

In [29]:
env.task._walker_features['body_positions'].flatten()[:10]

array([-0.16228435, -0.00131437,  0.10500313, -0.05942653,  0.10071903,
        0.12155984,  0.22211797,  0.20225642,  0.39432953,  0.38291664])

## Each Transformation Checking (transform function check)

In [30]:
obs = env.task._walker.transform_vec_to_egocentric_frame(env.physics,
                                                         (env.task._clip_reference_features['body_positions'][time_steps] - 
                                                          env.task._walker_features['body_positions'])[:, env.task._body_idxs])

obs_flattened = np.concatenate([o.flatten() for o in obs])
obs_flattened[:10]

array([ 7.01153267e-06,  3.45142537e-02, -1.07003822e-02,  3.81264065e-04,
        3.45874558e-02, -1.13416464e-02,  2.83746472e-02,  7.25784742e-03,
       -3.63179052e-02,  2.08227386e-02])

brax environment after transformation, seems not to be the same
```python
[-6.7749232e-02  8.6231794e-06 -1.7833591e-02 -1.0716664e-01
  3.4846365e-05 -2.3776948e-02  1.6838254e-01 -2.8296500e-02
  2.9513806e-02  8.9463890e-03]
```

In [31]:
def global_vector_to_local_frame(data, vec_in_world_frame):
    xmat = jp.reshape(data.xmat[0], (3, 3))
    if vec_in_world_frame.shape[-1] == 2:
      return jp.dot(vec_in_world_frame, xmat[:2, :2])
    elif vec_in_world_frame.shape[-1] == 3:
      return jp.dot(vec_in_world_frame, xmat)
    else:
      raise ValueError('`vec_in_world_frame` should have shape with final '
                       'dimension 2 or 3: got {}'.format(
                           vec_in_world_frame.shape))
    
def f(x):
      if len(x.shape) != 1:
        return jax.lax.dynamic_slice_in_dim(
          x, 
          state.info['cur_frame'] + 1, 
          env_brax._ref_traj_length, 
        )
      return jp.array([])

In [32]:
ref_traj = jax.tree_util.tree_map(f, env_brax._ref_traj)

xpos_flatten = state.pipeline_state.xpos[env_brax._body_locations].flatten()

obs = global_vector_to_local_frame(state.pipeline_state,
                             (ref_traj.body_positions - xpos_flatten)
                             .reshape([env_brax._ref_traj_length, 30, 3]))

# obs_flattened = jp.concatenate([o.flatten() for o in obs])

## Each Transformation Checking (differences check)

In [33]:
(env.task._clip_reference_features['body_positions'][time_steps] -
                  env.task._walker_features['body_positions'])[:, env.task._body_idxs].flatten()[:10]

array([-3.61171890e-02, -8.25232182e-06,  1.13160638e-03, -3.63955649e-02,
        3.64310787e-04,  5.48213873e-04, -1.87115775e-02,  2.82801469e-02,
       -3.20452392e-02,  6.30396974e-02])

In [34]:
(ref_traj.body_positions - xpos_flatten).reshape([env_brax._ref_traj_length, 30, 3]).flatten()[:10]

Array([-6.7749232e-02,  8.6231794e-06, -1.7833591e-02, -1.0716664e-01,
        3.4846365e-05, -2.3776948e-02,  1.6838254e-01, -2.8296500e-02,
        2.9513806e-02,  8.9463890e-03], dtype=float32)